Copyright (c) 2023, Douglas Santry
All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, is permitted provided that the following conditions are met:

1. Redistributions of source code must retain the above copyright notice, this
   list of conditions and the following disclaimer.
2. Redistributions in binary form must reproduce the above copyright notice,
   this list of conditions and the following disclaimer in the documentation
   and/or other materials provided with the distribution.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND
ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED
WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT OWNER OR CONTRIBUTORS BE LIABLE FOR
ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES
(INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES;
LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND
ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT
(INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS
SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

In [1]:
import tensorflow as tf
import numpy as np

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Softmax
from keras.layers import Conv2D as CNN
from keras.layers import MaxPooling2D as MaxPool
from keras.layers import Flatten
from tensorflow.keras.utils import to_categorical

import matplotlib.pyplot as plt
import math as math


In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

In [3]:
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5  # Normalize the images to [-1, 1]

In [4]:
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1).astype('float32')
test_images = (test_images - 127.5) / 127.5  # Normalize the images to [-1, 1]

In [5]:
HotEncodedVec = to_categorical (train_labels)

y = HotEncodedVec

In [6]:
def ModelFactory ():
    
    model = tf.keras.Sequential ()                                                 

    model.add (CNN (5, kernel_size=3, activation="relu", input_shape=(28,28,1)))
    model.add (MaxPool (pool_size=(2, 2), strides=(2,2), padding="valid"))

    model.add (Flatten ())

    model.add (Dense(50, activation='relu'))
    model.add (Dense(50, activation='relu'))

    model.add (Dense (10, activation='softmax'))

    model.compile (loss='categorical_crossentropy', optimizer='ADAM')

    return model

In [7]:
MNIST = ModelFactory ()

Metal device set to: Apple M1


2023-12-21 14:07:32.065176: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-21 14:07:32.065267: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
#
# Algorithm 5.6
#

ADAM = tf.keras.optimizers.Adam ()

In [9]:
#
# Algorithm 5.3, RPROP+
#

del_0 = 1e-2
del_min = 1e-8
del_max = 20         # What effect does this parameter have on MNIST convergence?

eta_plus = 1.2
eta_minus = 0.5

del_model = []
dL_model = []

for W in MNIST.trainable_variables:
    
    del_model.append (np.full (W.shape, del_0))
    dL_model.append (np.zeros (W.shape))


def RPROP (gradients):

    for i in range (len (MNIST.trainable_variables)):

        dL = gradients[i].numpy ()

        directions = np.sign (dL * dL_model[i])
        updates = np.sign (dL)

        case1 = directions > 0
        case2 = directions < 0
        case3 = directions == 0

        # Case 1
        del_model[i][case1] = np.minimum (del_model[i][case1] * eta_plus, del_max)
        updates[case1] = -updates[case1]

        # Case 2
        del_model[i][case2] = np.maximum (del_model[i][case2] * eta_minus, del_min)

        # Case 3
        updates[case3] = -updates[case3]

        # Apply the computed update and store new state
        MNIST.trainable_variables[i].assign_add (del_model[i] * updates)
        dL_model[i] = dL
        dL_model[i][case2] = 0.0

In [10]:
#
# Equation 3.40
#
# UpdateWeights in algorithm 3.3 (line 15 omitted as not required)
#

def UpdateWeights (grad):
    
    for i in range (len (grad)):
        MNIST.trainable_variables[i].assign_add (-0.01 * grad[i])

In [11]:
#
# Equation 4.7
#

MLE = tf.keras.losses.CategoricalCrossentropy (from_logits=False, reduction=tf.keras.losses.Reduction.NONE)


In [12]:
#
# A training loop.  There are 3 optimizers to choose from.  Only uncomment 1 at time.
#

BatchSize = 128

for epoch in range (1000):
    
    fake_SGD  = list (map (lambda x : int (x), np.random.uniform (0, len (train_images), BatchSize)))
    
    with tf.GradientTape () as tape:
    
        # Note that we are not using predict.  This is a slower path that the tape
        # can monitor.
        y_ = MNIST (train_images[fake_SGD], training=True)
    
        z = MLE (y_, y[fake_SGD])
        Loss = tf.reduce_sum(z.numpy ()) / BatchSize
    
    grad = tape.gradient (z, MNIST.trainable_variables)
    
    if epoch % 50 == 0:
        print ("Epoch %d: %f" % (epoch, Loss))

    if Loss < 0.11:
        break

#
# There are 3 possible, all mutually exclusive, update strategy choices.  Only uncomment one!
#

#    RPROP (grad)
#    ADAM.apply_gradients (zip (grad, MNIST.trainable_variables))
#    UpdateWeights (grad)


Epoch 0: 3405.246826


In [13]:
Z = MNIST.predict (test_images, verbose=False)

class_predictions = Z.argmax (1)

sum (class_predictions == test_labels) / len (test_labels) * 100 # Percent accurate

2023-12-21 14:07:39.433430: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-12-21 14:07:39.473895: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


9.8